# YOLOv12x: Fine-tuning and Evaluation

This notebook demonstrates how to:
1. Install required dependencies
2. Prepare dataset for training
3. Fine-tune YOLOv12x on a custom dataset
4. Run inference on test images
5. Calculate and visualize evaluation metrics

## 1. Install Required Dependencies

In [ ]:
# Install ultralytics package for YOLOv12
!pip install ultralytics
!pip install opencv-python matplotlib seaborn

# Check CUDA availability 
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Import Required Libraries

In [ ]:
import os
import yaml
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
from IPython.display import display, Image
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

## 3. Dataset Preparation

Let's assume we're working with a dataset that follows the YOLO format:
- images/ folder containing training images
- labels/ folder containing corresponding labels in YOLO format
- A YAML configuration file describing classes and dataset paths

If your dataset is structured differently, you'll need to adjust this section accordingly.

In [ ]:
# Define dataset paths - customize these for your specific project
DATASET_DIR = "../dataset_split"  # Change this!
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")
TEST_DIR = os.path.join(DATASET_DIR, "test")

# # Create dataset configuration YAML
# dataset_config = {
#     'path': DATASET_DIR,
#     'train': os.path.relpath(TRAIN_DIR, DATASET_DIR),
#     'val': os.path.relpath(VAL_DIR, DATASET_DIR),
#     'test': os.path.relpath(TEST_DIR, DATASET_DIR),
#     'names': {
#         # Add your class names and indices here
#         # For example:
#         # 0: 'car',
#         # 1: 'truck',
#         # 2: 'bus',
#         # ...
#     }
# }

# Write the dataset configuration to a YAML file
yaml_path = os.path.join(DATASET_DIR, "data.yaml")
# with open(yaml_path, 'w') as file:
#     yaml.dump(dataset_config, file)

# print(f"Dataset configuration saved to: {yaml_path}")

## 4. Fine-tuning YOLOv12x Model

Now we'll load a pre-trained YOLOv12x model and fine-tune it on our custom dataset.

In [ ]:
# Load pre-trained YOLOv12x model
model = YOLO('yolov12x.pt')

# Define training hyperparameters optimized for small dataset (~400 images)
hyperparameters = {
    'epochs': 100,          # More epochs for small dataset
    'batch': 8,             # Smaller batch size
    'imgsz': 640,           # Image size
    'patience': 20,         # Increased patience for early stopping
    'device': 0,            # Device to use (0 for first GPU)
    'workers': 4,           # Reduced worker threads
    'optimizer': 'AdamW',   # Optimizer
    'lr0': 0.0005,          # Lower initial learning rate
    'lrf': 0.01,            # Final learning rate factor
    'momentum': 0.937,      # SGD momentum
    'weight_decay': 0.001,  # Increased weight decay to prevent overfitting
    'warmup_epochs': 5.0,   # Longer warmup
    'warmup_momentum': 0.8, # Warmup momentum
    'warmup_bias_lr': 0.1,  # Warmup bias lr
    'box': 7.5,             # Box loss gain
    'cls': 0.5,             # Class loss gain
    'hsv_h': 0.015,         # Image HSV-Hue augmentation
    'hsv_s': 0.7,           # Image HSV-Saturation augmentation
    'hsv_v': 0.4,           # Image HSV-Value augmentation
    'translate': 0.2,       # Increased translation for better augmentation
    'scale': 0.6,           # Increased scale variation
    'fliplr': 0.5,          # Image flip left-right probability
    'flipud': 0.2,          # Add up-down flipping
    'mosaic': 1.0,          # Maximize mosaic augmentation
    'mixup': 0.15,          # Add mixup augmentation
    'copy_paste': 0.1,      # Add copy-paste augmentation
}

# Create model results directory
results_dir = os.path.join(os.getcwd(), "yolov12x_results")
os.makedirs(results_dir, exist_ok=True)

# Train the model
results = model.train(
    data=yaml_path,
    project=results_dir,
    name='fine_tuned_model',
    exist_ok=True,
    **hyperparameters
)

print(f"Training completed. Model saved to: {os.path.join(results_dir, 'fine_tuned_model')}")

## 5. Model Inference and Evaluation

Now, let's evaluate the fine-tuned model on the test set and calculate performance metrics.

In [ ]:
# Load the fine-tuned model
fine_tuned_model_path = os.path.join(results_dir, 'fine_tuned_model', 'weights', 'best.pt')
model = YOLO(fine_tuned_model_path)

# Run validation on the test set
test_results = model.val(
    data=yaml_path,
    split='test',  # Use the test split
    imgsz=640,
    batch=16,
    verbose=True,
    conf=0.25,    # Confidence threshold
    iou=0.5,      # IoU threshold
    project=results_dir,
    name='evaluation',
    exist_ok=True
)

print("Test results summary:")
print(f"mAP50: {test_results.box.map50:.5f}")
print(f"mAP50-95: {test_results.box.map:.5f}")
print(f"Precision: {test_results.box.mp:.5f}")
print(f"Recall: {test_results.box.mr:.5f}")

## 6. Detailed Analysis per Class

Let's analyze the model performance for each class separately.

In [ ]:
# Get class-wise metrics from the validation results
class_map = test_results.names  # Class index to name mapping
class_metrics = test_results.metrics.cls  # Class metrics
class_precisions = test_results.metrics.precision  # Class precisions
class_recalls = test_results.metrics.recall  # Class recalls

# Create a DataFrame for better visualization
metrics_df = pd.DataFrame({
    'Class': [class_map[i] for i in range(len(class_map))],
    'AP50': test_results.ap50_per_class,
    'AP50-95': test_results.ap_per_class,
    'Precision': class_precisions,
    'Recall': class_recalls
})

display(metrics_df)

# Plot AP50 for each class
plt.figure(figsize=(12, 6))
sns.barplot(x='Class', y='AP50', data=metrics_df)
plt.title('AP50 for Each Class')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 7. Confusion Matrix

The confusion matrix helps us see how well the model differentiates between different classes.

In [ ]:
# Plot confusion matrix
conf_matrix = test_results.confusion_matrix.matrix
plt.figure(figsize=(12, 10))
sns.heatmap(
    conf_matrix / np.sum(conf_matrix, axis=1)[:, None],  # Normalize by row (true classes)
    annot=True,
    fmt='.2f',
    cmap='Blues',
    xticklabels=list(class_map.values()),
    yticklabels=list(class_map.values())
)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix')
plt.tight_layout()
plt.show()

## 8. Visualizing Detection Results on Test Images

Let's visualize some predictions on test images:

In [ ]:
# Get list of test images
test_images_dir = os.path.join(TEST_DIR, 'images')
test_images = list(Path(test_images_dir).glob('*.jpg')) + list(Path(test_images_dir).glob('*.png'))
test_images = [str(img) for img in test_images]

# Select random images for visualization
if len(test_images) > 0:
    sample_images = random.sample(test_images, min(5, len(test_images)))
    
    for img_path in sample_images:
        # Run inference
        results = model(img_path, conf=0.25)
        
        # Display results
        for result in results:
            fig, ax = plt.subplots(1, 1, figsize=(12, 9))
            img = result.orig_img
            
            # Plot detections
            for box, conf, cls in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
                x1, y1, x2, y2 = box.cpu().numpy().astype(int)
                class_id = int(cls.item())
                class_name = class_map[class_id]
                confidence = conf.item()
                
                # Draw bounding box
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Add label
                label = f"{class_name}: {confidence:.2f}"
                cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax.set_title(f"Predictions on {os.path.basename(img_path)}")
            ax.axis("off")
            plt.tight_layout()
            plt.show()
else:
    print("No test images found")

## 9. Precision-Recall Curves

Let's plot precision-recall curves for each class:

In [ ]:
# Create P-R curve plots for each class
plt.figure(figsize=(12, 8))

# Get P-R curve data from results
precision_data = test_results.curves[0].data
recall_data = test_results.curves[1].data

# Plot P-R curves for each class
for i in range(len(class_map)):
    plt.plot(recall_data[:, i], precision_data[:, i], label=f'{class_map[i]}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.legend(loc='lower left')
plt.grid(True)
plt.show()

## 10. Export Model for Deployment

Let's save our fine-tuned model in different formats for deployment.

In [ ]:
# Export model to different formats
export_path = os.path.join(results_dir, "exported_models")
os.makedirs(export_path, exist_ok=True)

# Export to ONNX format
model.export(format="onnx", imgsz=640)

# Export to TorchScript format
model.export(format="torchscript", imgsz=640)

print(f"Models exported to {export_path}")
print("Available formats:")
for file in os.listdir(export_path):
    print(f"- {file}")

## 11. Comparative Analysis with Previous YOLO Versions

Let's compare YOLOv12x with previous YOLO versions to highlight improvements:

In [ ]:
# Load performance metrics from other models (if available)
comparison_metrics = {
    'Model': ['YOLOv8x', 'YOLOv11x', 'YOLOv12x'],
    'mAP50': [0.0, 0.0, 0.0],  # Placeholder values
    'mAP50-95': [0.0, 0.0, 0.0],
    'Precision': [0.0, 0.0, 0.0],
    'Recall': [0.0, 0.0, 0.0],
    'Inference Time (ms/img)': [0.0, 0.0, 0.0]
}

# Try to load actual metrics if available
try:
    # YOLOv8 metrics
    yolov8_results_path = "../yolov8x_results/evaluation"
    if os.path.exists(yolov8_results_path):
        # Get the metrics (this is just a placeholder, actual implementation depends on how metrics are stored)
        comparison_metrics['mAP50'][0] = 0.85  # Example value
        comparison_metrics['mAP50-95'][0] = 0.65  # Example value
        comparison_metrics['Precision'][0] = 0.82  # Example value
        comparison_metrics['Recall'][0] = 0.80  # Example value
        comparison_metrics['Inference Time (ms/img)'][0] = 12.5  # Example value
    
    # YOLOv11 metrics
    yolov11_results_path = "../yolov11x_results/evaluation"
    if os.path.exists(yolov11_results_path):
        # Get the metrics (this is just a placeholder, actual implementation depends on how metrics are stored)
        comparison_metrics['mAP50'][1] = 0.87  # Example value
        comparison_metrics['mAP50-95'][1] = 0.67  # Example value
        comparison_metrics['Precision'][1] = 0.84  # Example value
        comparison_metrics['Recall'][1] = 0.81  # Example value
        comparison_metrics['Inference Time (ms/img)'][1] = 11.8  # Example value
    
    # Current YOLOv12 metrics
    comparison_metrics['mAP50'][2] = test_results.box.map50
    comparison_metrics['mAP50-95'][2] = test_results.box.map
    comparison_metrics['Precision'][2] = test_results.box.mp
    comparison_metrics['Recall'][2] = test_results.box.mr
    # You'd need to calculate inference time separately
    comparison_metrics['Inference Time (ms/img)'][2] = 11.0  # Example value
    
    # Create DataFrame and visualize comparison
    comparison_df = pd.DataFrame(comparison_metrics)
    display(comparison_df)
    
    # Plot comparison metrics
    metrics_to_plot = ['mAP50', 'mAP50-95', 'Precision', 'Recall']
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metrics_to_plot):
        plt.subplot(2, 2, i+1)
        sns.barplot(x='Model', y=metric, data=comparison_df)
        plt.title(f'Comparison of {metric}')
        plt.ylabel(metric)
        plt.ylim([0, 1])
        
    plt.tight_layout()
    plt.show()
    
    # Plot inference time comparison
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Model', y='Inference Time (ms/img)', data=comparison_df)
    plt.title('Inference Speed Comparison')
    plt.ylabel('Time (ms/image)')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Couldn't load comparison metrics: {e}")
    print("Run all model evaluations first to enable comparison.")

## 12. Summary and Conclusion

We have successfully:
1. Fine-tuned YOLOv12x on a custom dataset
2. Evaluated its performance on the test set
3. Analyzed per-class metrics and visualized results
4. Exported the model for deployment
5. Compared performance with earlier YOLO versions

Key metrics:
- mAP50: How accurate the model is at IoU threshold of 0.5
- mAP50-95: How accurate the model is across multiple IoU thresholds
- Precision: How many of the predicted detections are correct
- Recall: How many of the ground truth objects are detected

Improvements in YOLOv12x compared to earlier versions:
- [List key improvements based on your comparative analysis]

To improve results further, consider:
- Increasing the number of training epochs
- Adding more training data or using data augmentation
- Adjusting hyperparameters like learning rate and batch size
- Using different model variants (YOLOv12s, YOLOv12m, etc.)